# Retrieval Agumented Generation

In [5]:
import faiss
import transformers
from sentence_transformers import SentenceTransformer

In [6]:
embed_model = SentenceTransformer("nomic-ai/nomic-embed-text-v1", trust_remote_code=True)
sentences = ['search_query: What is TSNE?', 'search_query: Who is Laurens van der Maaten?']
embeddings = embed_model.encode(sentences)
print(embeddings)

You try to use a model that was created with version 2.4.0.dev0, however, your version is 2.3.1. This might cause unexpected behavior or errors. In that case, try to update to the latest version.





pytorch_model.bin:   0%|          | 0.00/547M [00:00<?, ?B/s]